<a href="https://colab.research.google.com/github/zahra75koosha/zahra-koosha/blob/master/name_entity_recognition_project.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
!pip3 uninstall tensorflow
!pip3 install tensorflow.gpu==2.0.0

In [ ]:
!pip install hazm

In [ ]:
from google.colab import drive
drive.mount('/content/drive')


In [4]:
%cd /content/drive/My Drive/
%ls

/content/drive/My Drive
'Colab Notebooks'/   dev.txt   test.txt   train.txt


In [5]:
import pandas as pd
import sklearn 
import sys, os, re, codecs
import gensim.models.keyedvectors as word2vec

from gensim.models import KeyedVectors

from tensorflow.keras import optimizers
from tensorflow.keras.models import Model, Sequential
from tensorflow.keras.layers import Dense, Input, Embedding, Dropout, GlobalMaxPool1D
from keras.layers import GRU, SimpleRNN
from keras import initializers, regularizers, constraints, optimizers, layers

import matplotlib.pyplot as plt
# %matplotlib inline

from tensorflow.keras.layers import LSTM, Bidirectional
from tensorflow.keras.utils import to_categorical
from tensorflow.keras.preprocessing.text import Tokenizer
from tensorflow.keras.preprocessing.sequence import pad_sequences

from hazm import *

Using TensorFlow backend.


In [179]:
train_data = open("/content/drive/My Drive/train.txt")
dev_data= open("/content/drive/My Drive/dev.txt")

In [180]:
print(train_data.read(100))

به O
عنوان O
مثال O
وقتی O
نشریات O
مدافع O
اصول O
و O
ارزشها O
و O
منادی O
انقلاب O
و O
اسلام O
در 


In [182]:
# convert dataset to dataframe type

list_columns = ['sentence','label']

def read_data(dataset):
    df= pd.DataFrame(columns=list_columns)
    
    file = open(dataset)
    sentences = []
    sentence = []
    for line in file:
        if len(line) == 0 or line.startswith('-DOCSTART') or line[0] == "\n":
            if len(sentence) > 0:
                sentences.append(sentence)
                sentence = []
            continue
        splits = line.split(' ')        
        sentence.append([splits[0], splits[-1]])

    if len(sentence) > 0:
        sentences.append(sentence)
        sentence = []
        
    for sentence in sentences:
      text=''
      words=[]
      values=[]
      for item in sentence:
        words.append(item[0])
        values.append(item[1].replace('\n',''))
      _list=[]
      _list.append(' '.join(words))
      _list.append(' '.join(values))
      df2 = pd.DataFrame([_list], columns=list_columns)
      df=df.append(df2, ignore_index=True)
    return df

In [183]:
train_data_set = read_data("/content/drive/My Drive/train.txt")

In [184]:
train_data_set


,sentence,label
0,به عنوان مثال وقتی نشریات مدافع اصول و ارزشها ...,O O O O O O O O O O O O O O O O O O O O O O O ...
1,دکتر اصغری دبیر چهارمین همایش انجمن زمین‌شناسی...,O B-pers O B-event I-event I-event I-event I-e...
2,دکتر اکبر میرعرب در همایش بررسی و پیشگیری از ب...,O B-pers I-pers O B-event I-event I-event I-ev...
3,اردبیل ـ استاندار اردبیل گفت : به مناسبت هفته ...,B-loc O O B-loc O O O O B-event I-event O O O ...
4,حمید طاهایی افزود : برای اجرای این طرحها 0 میل...,B-pers I-pers O O O O O O O O O O O O O O O O O
...,...,...
10236,می‌خواهم بچه داشته باشم ولی نه در اینجا .,O O O O O O O O O
10237,می‌دانم با رفتنم همه را از دست خواهم داد ولی ب...,O O O O O O O O O O O O O
10238,حدود هفتاد و پنج نفر در یک اتاق کوچک جمع شده ب...,O O O O O O O O O O O O O
10239,داستان زندگی و مهاجرت آتی او به نمایش گذاشته شد .,O O O O O O O O O O O


In [185]:
dev_data_set = read_data("/content/drive/My Drive/dev.txt")

In [186]:
dev_data_set

,sentence,label
0,وي افزود : کمیته کشوری مبارزه با بیماری ایدز د...,O O O O O O O O O O O O O O O O O O O O O O O ...
1,دکتر اکبر میرعرب در همایش بررسی و پیشگیری از ب...,O B-pers I-pers O B-event I-event I-event I-ev...
2,حمید طاهایی افزود : برای اجرای این طرحها 0 میل...,B-pers I-pers O O O O O O O O O O O O O O O O O
3,طاهایی گفت : 0 طرح عمرانی ، 0 طرح مدرسه‌سازی د...,B-pers O O O O O O O O O O O O O O O O O O O O...
4,تل‌مش تپه‌ای باستانی متعلق به هزاره چهارم پیش ...,B-loc O O O O O O O O O O O O O O O B-loc I-lo...
...,...,...
5115,اگر اینجا بمانم ، خفه خواهم شد .,O O O O O O O O
5116,هر لحظه از زندگیم را تا هنگام مرگ می‌توانم ، پ...,O O O O O O O O O O O O O
5117,ویکا چند شب قبل از عزیمت به اسرائیل به اتفاق م...,B-pers O O O O O O B-loc O O O O O O O O O O O...
5118,دوستان ویکا به جای عروسک صحبت می‌کردند .,O B-pers O O O O O O


In [187]:
type(dev_data_set)

pandas.core.frame.DataFrame

In [188]:
train_sentence = train_data_set['sentence']
train_label = train_data_set['label']

In [189]:
dev_sentence = dev_data_set['sentence']
dev_labels = dev_data_set['label']

In [190]:
train_sentence[:10]

0    به عنوان مثال وقتی نشریات مدافع اصول و ارزشها ...
1    دکتر اصغری دبیر چهارمین همایش انجمن زمین‌شناسی...
2    دکتر اکبر میرعرب در همایش بررسی و پیشگیری از ب...
3    اردبیل ـ استاندار اردبیل گفت : به مناسبت هفته ...
4    حمید طاهایی افزود : برای اجرای این طرحها 0 میل...
5    اصفهان ـ 0 تپه و محوطه باستانی دوره‌های تاریخی...
6    پیشینه این محوطه‌های باستانی ، مربوط به پیش از...
7    تل‌مش تپه‌ای باستانی متعلق به هزاره چهارم پیش ...
8    محسن جاوری ، سرپرست هیأت باستان‌شناسی این مناط...
9    میل سنگی دوره ساسانی با ارتفاع 0 سانتیمتر و غا...
Name: sentence, dtype: object

In [191]:
train_label[:10]

0    O O O O O O O O O O O O O O O O O O O O O O O ...
1    O B-pers O B-event I-event I-event I-event I-e...
2    O B-pers I-pers O B-event I-event I-event I-ev...
3    B-loc O O B-loc O O O O B-event I-event O O O ...
4      B-pers I-pers O O O O O O O O O O O O O O O O O
5    B-loc O O O O O O O O O O O O O O B-loc I-loc ...
6                    O O O O O O O O O O O O O O O O O
7    B-loc O O O O O O O O O O O O O O O B-loc I-lo...
8    B-pers I-pers O O O O O O O O O O O O O O O O ...
9    B-pro I-pro O O O O O O O O O O O O B-loc I-lo...
Name: label, dtype: object

In [195]:
def label_to_one_hot(value):
  list_classes = ['B-pers','I-pers', 'B-pro', 'I-pro','B-loc','I-loc','B-fac','I-fac','B-event','I-event','B-org','I-org','O']
  if value == list_classes[0]:
    return [1, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0]
  if value == list_classes[1]:
    return [0, 1, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0]
  if value == list_classes[2]:
    return [0, 0, 1, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0]
  if value == list_classes[3]:
    return [0, 0, 0, 1, 0, 0, 0, 0, 0, 0, 0, 0, 0]
  if value == list_classes[4]:
    return [0, 0, 0, 0, 1, 0, 0, 0, 0, 0, 0, 0, 0]
  if value == list_classes[5]:
    return [0, 0, 0, 0, 0, 1, 0, 0, 0, 0, 0, 0, 0]
  if value == list_classes[6]:
    return [0, 0, 0, 0, 0, 0, 1, 0, 0, 0, 0, 0, 0]
  if value == list_classes[7]:
    return [0, 0, 0, 0, 0, 0, 0, 1, 0, 0, 0, 0, 0]
  if value == list_classes[8]:
    return [0, 0, 0, 0, 0, 0, 0, 0, 1, 0, 0, 0, 0]
  if value == list_classes[9]:
    return [0, 0, 0, 0, 0, 0, 0, 0, 0, 1, 0, 0, 0]
  if value == list_classes[10]:
    return [0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 1, 0, 0]
  if value == list_classes[11]:
    return [0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 1, 0]
  if value == list_classes[12]:
    return [0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 1]

In [196]:
max_features = 250015
tokenizer = Tokenizer(num_words=max_features)
tokenizer.fit_on_texts(list(train_sentence))
tokenized_train = tokenizer.texts_to_sequences(train_sentence)

train_labels = [[label_to_one_hot(c) for c in ey.split(' ')] for ey in train_label]

In [199]:
tokenized_train[4]

[2942, 8677, 65, 12, 234, 6, 1507, 10, 188, 1, 10, 125, 253, 535, 449, 21, 9]

In [200]:
train_labels[4]

[[1, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0],
 [0, 1, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0],
 [0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 1],
 [0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 1],
 [0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 1],
 [0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 1],
 [0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 1],
 [0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 1],
 [0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 1],
 [0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 1],
 [0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 1],
 [0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 1],
 [0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 1],
 [0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 1],
 [0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 1],
 [0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 1],
 [0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 1],
 [0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 1],
 [0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 1]]

In [205]:
# Find maximum length of training sentences
max_length = max([len(s) for s in tokenized_train])
max_length


238

In [208]:
# Pad embeded training sequences
x_train_padded = pad_sequences(tokenized_train, maxlen=max_length, padding='post')
y_train_padded = pad_sequences(train_labels, maxlen=max_length, padding='post')

In [211]:
x_train_padded[4]

array([2942, 8677,   65,   12,  234,    6, 1507,   10,  188,    1,   10,
        125,  253,  535,  449,   21,    9,    0,    0,    0,    0,    0,
          0,    0,    0,    0,    0,    0,    0,    0,    0,    0,    0,
          0,    0,    0,    0,    0,    0,    0,    0,    0,    0,    0,
          0,    0,    0,    0,    0,    0,    0,    0,    0,    0,    0,
          0,    0,    0,    0,    0,    0,    0,    0,    0,    0,    0,
          0,    0,    0,    0,    0,    0,    0,    0,    0,    0,    0,
          0,    0,    0,    0,    0,    0,    0,    0,    0,    0,    0,
          0,    0,    0,    0,    0,    0,    0,    0,    0,    0,    0,
          0,    0,    0,    0,    0,    0,    0,    0,    0,    0,    0,
          0,    0,    0,    0,    0,    0,    0,    0,    0,    0,    0,
          0,    0,    0,    0,    0,    0,    0,    0,    0,    0,    0,
          0,    0,    0,    0,    0,    0,    0,    0,    0,    0,    0,
          0,    0,    0,    0,    0,    0,    0,   

In [210]:
y_train_padded[4]

array([[1, 0, 0, ..., 0, 0, 0],
       [0, 1, 0, ..., 0, 0, 0],
       [0, 0, 0, ..., 0, 0, 1],
       ...,
       [0, 0, 0, ..., 0, 0, 0],
       [0, 0, 0, ..., 0, 0, 0],
       [0, 0, 0, ..., 0, 0, 0]], dtype=int32)

In [212]:
#vocabulary size
vocab_size = len(tokenizer.word_index)

In [213]:
model_blstm = Sequential()

model_blstm.add(Embedding(vocab_size, 300, input_length=max_length))
model_blstm.add(Bidirectional(LSTM(300, return_sequences=True, name='lstm_layer')))

model_blstm.add(GlobalMaxPool1D())
model_blstm.add(Dropout(0.25))
model_blstm.add(Dense(300, activation="relu"))
model_blstm.add(Dropout(0.2))
model_blstm.add(Dense(5, activation='softmax'))


In [214]:
model_blstm.compile(loss='categorical_crossentropy',
              optimizer='adam',
              metrics=["categorical_accuracy"])

model_blstm.summary()
batch_size_blstm = 64
epochs_blstm = 12

Model: "sequential"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
embedding (Embedding)        (None, 238, 300)          5462400   
_________________________________________________________________
bidirectional (Bidirectional (None, 238, 600)          1442400   
_________________________________________________________________
global_max_pooling1d (Global (None, 600)               0         
_________________________________________________________________
dropout (Dropout)            (None, 600)               0         
_________________________________________________________________
dense (Dense)                (None, 300)               180300    
_________________________________________________________________
dropout_1 (Dropout)          (None, 300)               0         
_________________________________________________________________
dense_1 (Dense)              (None, 5)                 1

In [215]:
hist_blstm = model_blstm.fit(x_train_padded, y_train_padded,
                             batch_size=batch_size_blstm, epochs=epochs_blstm,
                             shuffle=True)

ValueError: ignored

In [128]:
'''import numpy as np
# Convert dataframes to numpy arrays
train_sentence = np.asarray(train_sentence)
train_label = np.asarray(train_label)'''

'import numpy as np\n# Convert dataframes to numpy arrays\ntrain_sentence = np.asarray(train_sentence)\ntrain_label = np.asarray(train_label)'

In [175]:

'''import re
normalizer = Normalizer()

# turn a doc into clean tokens
def clean_data(doc):
    doc = normalizer.normalize(doc) # Normalize document using Hazm Normalizer
    #tokenized = word_tokenize(doc)  # Tokenize text
    

    tokens = []
    for token in doc:
      token = re.sub("[،:.,;()/+]", " ", token) 
    
      tokens.append(token)


    return tokens'''

In [178]:
'''# Apply preprocessing step to training data
words = np.empty_like(train_sentence)
for index, document in enumerate(train_sentence):
  words[index] = clean_data(document)'''

AttributeError: ignored